In [1]:
import requests
import csv
import os
from datetime import datetime

# Define API parameters
API_ENDPOINT = "https://v6.exchangerate-api.com/v6/{api_key}/latest/USD"
API_KEY = "" # API GOES HERE  
CURRENCIES = ["EUR", "GBP", "INR", "NPR"]
CSV_FILE = "exchange_rates.csv"

# Fetch exchange rates
def fetch_exchange_rates(api_endpoint, api_key):
    url = api_endpoint.format(api_key=api_key)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch exchange rates: {response.status_code}")
        return None

# Write exchange rates to CSV
def write_to_csv(data, filename):
    file_exists = os.path.exists(filename)
    with open(filename, mode="a" if file_exists else "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write headers if file doesn't exist
        if not file_exists:
            writer.writerow(["Date", "Currency", "Exchange Rate"])
        # Write exchange rates
        for currency, rate in data.items():
            writer.writerow([datetime.now().strftime("%Y-%m-%d"), currency, rate])

# Compare today's rates with historical rates
def compare_rates(filename, latest_rates):
    if not os.path.exists(filename):
        print("No historical data to compare.")
        return

    historical_data = {}
    # Read historical rates from CSV
    with open(filename, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["Currency"] in latest_rates:
                historical_data[row["Currency"]] = float(row["Exchange Rate"])

    # Compare and calculate differences
    differences = {}
    for currency, latest_rate in latest_rates.items():
        historical_rate = historical_data.get(currency)
        if historical_rate is not None:
            differences[currency] = latest_rate - historical_rate

    # Display the differences
    for currency, diff in differences.items():
        print(f"{currency}: {'+' if diff >= 0 else ''}{diff:.4f}")

# Main execution
if __name__ == "__main__":
    exchange_data = fetch_exchange_rates(API_ENDPOINT, API_KEY)
    if exchange_data and "conversion_rates" in exchange_data:
        # Extract relevant rates
        latest_rates = {currency: exchange_data["conversion_rates"].get(currency) for currency in CURRENCIES}
        print("Latest exchange rates:", latest_rates)

        # Save to CSV
        write_to_csv(latest_rates, CSV_FILE)
        print(f"Exchange rates saved to {CSV_FILE}.")

        # Compare with historical rates
        print("\nComparing with historical rates:")
        compare_rates(CSV_FILE, latest_rates)
    else:
        print("Failed to retrieve exchange rates.")


Latest exchange rates: {'EUR': 0.9525, 'GBP': 0.7869, 'INR': 84.9667, 'NPR': 135.9436}
Exchange rates saved to exchange_rates.csv.

Comparing with historical rates:
EUR: +0.0000
GBP: +0.0000
INR: +0.0000
NPR: +0.0000
